In [1]:
import os
import mypackages.myrasters as mr
import numpy as np
%matplotlib inline

In [2]:
data_dir = os.path.join('..', 'data')
out_dir = os.path.join('..', 'output/irrigation_prepared')

Read irrigation rasters:

In [3]:
aei_raster = mr.MyRaster(os.path.join(data_dir, 'FAO/Aquastat/gmia_v5_aei_pct.asc'))
aai_pct_aei_raster = mr.MyRaster(os.path.join(data_dir, 'FAO/Aquastat/gmia_v5_aai_pct_aei.asc'))

In [4]:
aei_raster.show_raster_properties()

Driver:  AAIGrid
Projection:  
----------------------------------------------------------------------------------------------------
Size:  4320 x 2160 x 1
X min =  -180.0
X max =  180.0
Y min =  -90.0
Y max =  90.0
Origin = ( -180.0 , 90.0 )
Pixel Size = ( 0.08333333333333333 , -0.08333333333333333 )
No Data Value =  -9.0


Calculate area actually irrigated as % of area:

In [5]:
aei = aei_raster.get_array()  # area equipped for irrigation in %
aai_pct_aei = aai_pct_aei_raster.get_array()  # area actually irrigated as % of area equipped for irrigation

In [6]:
aai = aei * aai_pct_aei / 100  # area actually irrigated as % of area
aai[aei == aei_raster.NoDataValue] = aei_raster.NoDataValue

Crop to size of soilgrid layers:

In [7]:
file_name = 'soilgrids/CLYPPT_M_sl1_250m_ll.tif'

in_filename = os.path.join(data_dir, file_name)
soilgrid_raster = mr.MyRaster(in_filename)
soilgrid_raster.show_raster_properties()

Driver:  GTiff
Projection:  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]
----------------------------------------------------------------------------------------------------
Size:  172800 x 67200 x 1
X min =  -180.0
X max =  179.99994240015582
Y min =  -56.00081039406058
Y max =  83.99916720600001
Origin = ( -180.0 , 83.99916720600001 )
Pixel Size = ( 0.0020833330000009018 , -0.0020833330000009018 )
No Data Value =  255.0


In [8]:
aai_cut = mr.cut_array_yboundaries(aai, -90, 90, -56.00081039406058, 83.99916720600001, -0.08333333333333333)

In [9]:
out_filename = 'area_actually_irrigated[%]'

mr.array2geotiff(aai_cut, os.path.join(out_dir, out_filename),
                 aei_raster.NoDataValue, aai_cut.shape[1], aai_cut.shape[0], 
                 -180, 83.99916720600001, aei_raster.pixelWidth, aei_raster.pixelHeight)


Resample irrigation raster to a 10 times higher resolution (resolution of soil water data):

In [10]:
%%time

scaling_factor = 10
aai_resampled = mr.resample_array_to_higher_resolution(aai_cut, scaling_factor)


CPU times: user 14.5 s, sys: 11.6 s, total: 26.1 s
Wall time: 26.1 s


In [11]:
aai_raster = mr.MyRaster(os.path.join(out_dir, 'area_actually_irrigated[%].tif'))

In [12]:
%%time

mr.array2geotiff(aai_resampled, os.path.join(out_dir, 'aai_resampled'), -9,
                 aai_raster.xSize * scaling_factor, aai_raster.ySize * scaling_factor,
                 aai_raster.xMin, aai_raster.yMax,
                 aai_raster.pixelWidth/scaling_factor, aai_raster.pixelHeight/scaling_factor)

CPU times: user 23 s, sys: 1.51 s, total: 24.5 s
Wall time: 24.3 s
